**Install Required Libraries**


In [1]:
!pip install transformers datasets sacrebleu evaluate fastapi uvicorn nest-asyncio --quiet
!pip install git+https://github.com/huggingface/accelerate --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s e

**Load MarianMT Model**

In [2]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-hi-en'  # Hindi to English
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/304M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

**Load Dataset (Custom idioms dataset)**

In [3]:
train_data = [
    {"hi": "रस्सी जल गयी, बल नहीं गया", "en": "Even after one hits rock bottom, their arrogance remains unchanged."},
    {"hi": "नाच ना जाने आँगन टेढ़ा", "en": "A bad workman blames his tools."},
    # Add more idioms here
]

from datasets import Dataset
train_dataset = Dataset.from_list(train_data)


**Tokenize & Format Dataset**

In [4]:
def preprocess(example):
    inputs = tokenizer(example['hi'], return_tensors='pt', padding="max_length", truncation=True, max_length=128)
    labels = tokenizer(example['en'], return_tensors='pt', padding="max_length", truncation=True, max_length=128)
    inputs['labels'] = labels['input_ids']
    return inputs

tokenized_dataset = train_dataset.map(preprocess, batched=True)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

**Fine-Tune the Model**

In [5]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=2,
    save_strategy="no",
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/tmp/ipython-input-5-3800456733.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 1ashutoshsingh2022 (1ashutoshsingh2022-na) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
2,9.985600
4,5.783500
6,3.901000
8,2.694800
10,2.061600


TrainOutput(global_step=10, training_loss=4.88529052734375, metrics={'train_runtime': 494.9999, 'train_samples_per_second': 0.04, 'train_steps_per_second': 0.02, 'total_flos': 677967298560.0, 'train_loss': 4.88529052734375, 'epoch': 10.0})

**Evaluate using BLEU/METEOR**

In [6]:
import evaluate
bleu = evaluate.load("bleu")

predictions = []
references = []

for item in train_data:
    input_ids = tokenizer(item["hi"], return_tensors="pt", truncation=True, padding=True).input_ids
    output_ids = model.generate(input_ids)[0]
    pred = tokenizer.decode(output_ids, skip_special_tokens=True)
    predictions.append(pred)
    references.append([item["en"]])

bleu_score = bleu.compute(predictions=predictions, references=references)
print("BLEU Score:", bleu_score)


BLEU Score: {'bleu': 0.0, 'precisions': [0.06896551724137931, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.5263157894736843, 'translation_length': 29, 'reference_length': 19}


**Create and Run a FastAPI Server**

In [8]:
from fastapi import FastAPI
import nest_asyncio
import uvicorn

app = FastAPI()
nest_asyncio.apply()

@app.get("/translate")
def translate_hi_to_en(text: str):
    input_ids = tokenizer(text, return_tensors="pt", padding=True, truncation=True).input_ids
    output = model.generate(input_ids)
    translation = tokenizer.decode(output[0], skip_special_tokens=True)
    return {"translation": translation}

uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [615]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [615]


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print(public_url)


**Add Gradio UI for Translation Testing**

In [9]:
!pip install transformers gradio datasets evaluate --quiet


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

**Load the fine-tuned MarianMT model**

In [10]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-hi-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)  # Replace with fine-tuned model if saved


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


**Define Translation Function**

In [11]:
def translate_hi_to_en(text):
    if not text.strip():
        return "Please enter a valid Hindi sentence."
    input_ids = tokenizer(text, return_tensors="pt", padding=True, truncation=True).input_ids
    output_ids = model.generate(input_ids)
    translation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return translation


**Create Gradio UI**

In [12]:
import gradio as gr

iface = gr.Interface(
    fn=translate_hi_to_en,
    inputs=gr.Textbox(lines=3, placeholder="Enter a Hindi sentence or idiom..."),
    outputs=gr.Textbox(label="Translated English"),
    title="Hindi Idiom Translator",
    description="Fine-tuned model for culturally nuanced Hindi to English translations.",
    theme="soft"
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b6fa64f1f0660983f4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Full  Working Hindi To English Idiom Translator**

In [16]:

!pip install transformers datasets gradio sacrebleu evaluate --quiet


from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
import torch
import gradio as gr


idioms_data = [
    {
        "hi": "रस्सी जल गयी, बल नहीं गया",
        "en": "Even after one hits rock bottom, their arrogance remains unchanged."
    },
    {
        "hi": "नाच ना जाने आँगन टेढ़ा",
        "en": "A bad workman blames his tools."
    },
    {
        "hi": "ऊँट के मुँह में जीरा",
        "en": "A drop in the ocean."
    }
]

dataset = Dataset.from_list(idioms_data)


model_name = 'Helsinki-NLP/opus-mt-hi-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


def preprocess(example):
    model_inputs = tokenizer(example["hi"], max_length=64, truncation=True, padding="max_length")
    labels = tokenizer(example["en"], max_length=64, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = dataset.map(preprocess)


training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=30,
    logging_steps=1,
    save_strategy="no",
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
)




/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/tmp/ipython-input-16-1835150983.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
1,9.929000
2,9.643500
3,6.783300
4,5.627900
5,4.400600
6,4.613100
7,2.802100
8,3.547400
9,2.236300
10,1.575000


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://995cb75a8c52ab1aad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
trainer.train()

# Step 7: Gradio UI
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).input_ids
    output = model.generate(inputs, max_length=64)
    return tokenizer.decode(output[0], skip_special_tokens=True)

iface = gr.Interface(
    fn=translate,
    inputs=gr.Textbox(lines=2, label="Enter Hindi idiom"),
    outputs="text",
    title="🧠 Hindi Idiom Translator",
    description="Fine-tuned model that understands Indian idioms",
)

iface.launch(share=True)